In [29]:
import pandas as pd
df = pd.read_pickle('yonhap_ready_2.pkl')

In [30]:
df.head(3)

,article,title,article_len,sec1,sec_idx
0,'회장이 청부폭력' 루머 협박범도 기소…사이버명예훼손 전담수사 첫 사례 ...,"""해경이 가만있으라 방송"" 허위사실 유포 40대 기소",966,사회,4
1,지난 해 9월 북한의 함경북도 항구도시 나진과 러시아 극동지역 도시 ...,南 점검단 방북…나진-하산 프로젝트 시범운송 시작(종합),1539,정치,6
2,포항 도착한 화물선 러시아 시베리아산 유연탄 4만 500t을...,나진-하산 프로젝트 탄력…시범수송 석탄 포항 도착(종합),1827,경제,0


In [31]:
X = df.article
y = df.sec_idx
print(X.shape)
print(y.shape)

(443312,)
(443312,)


In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(376815,)
(66497,)
(376815,)
(66497,)


In [33]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.63 ms


### Summary:
    * vect.fit(train) learns the vocabulary of the training data
    * vect.transform(train) uses the fitted vocabulary to build a document term matrix from the training data
    * vect.transform(test) uses the fitted vocabulary to build a document term matrix from the testing data(and ignores tokens it hasn't seen before)

In [34]:
stopwords_news=['지난해', '중','말', '뒤', '곳', '군', '위', '개', '간', '건', '이날', '도', '등', '명', '시', '앞', '원', '분', '회', '년', '것', '씨', '일', '월','오전','오후']

In [36]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(sublinear_tf=True, tokenizer=noun_tagger,stop_words=stopwords_news, ngram_range=(1,3), min_df=5, max_df=.5)
X_train_dtm = vect.fit_transform(X_train)

CPU times: user 26min 40s, sys: 18.2 s, total: 26min 58s
Wall time: 26min 57s


In [37]:
X_test_dtm = vect.transform(X_test)

In [38]:
X_train_dtm.shape

(376815, 2473920)

In [39]:
X_test_dtm.shape

(66497, 2473920)

## save vector & dtm

https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python
https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn

In [40]:
import pickle

with open("yonhap_tfidf_vect.pkl", 'wb') as handle:
    pickle.dump(vect, handle)

with open("yonhap_dtm.pkl", 'wb') as handle:
    pickle.dump(X_test_dtm, handle)

In [41]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.001)

In [42]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 3.56 s, sys: 284 ms, total: 3.84 s
Wall time: 3.85 s


MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [43]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [44]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.87316721055085189

In [45]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.94      0.90      0.92     16783
         과학       0.48      0.79      0.60       371
         국제       0.83      0.94      0.88     12136
         문화       0.77      0.90      0.83      3665
         사회       0.88      0.83      0.85     17451
         생활       0.51      0.76      0.61      1256
         정치       0.92      0.85      0.88     14835

avg / total       0.88      0.87      0.88     66497



In practice Multinomial NB can be better than Linear SVM in some situations and svm better than NB on other datasets. Apparently t's possible to combine the two approaches to get a very good baseline: projects:nbsvm - Sida I. Wang

You can also try non-linear SVMs but often the quadratic complexity of the SMO algorithm (for instance as implemented in libsvm) makes it not practical on datasets with more than 5000 documents. Instead on prefer to use liblinear than can only train linear SVM on large datasets.

https://www.quora.com/What-are-the-differences-similarities-between-SVM-Naive-Bayes-for-binary-text-classification-wrt-how-they-are-processing-the-features

http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [46]:
from sklearn import svm
#clf = svm.SVC(decision_function_shape='ovo')
clfrSVM=svm.LinearSVC(C=1)

In [47]:
%time clfrSVM.fit(X_train_dtm, y_train)

CPU times: user 1min 40s, sys: 624 ms, total: 1min 40s
Wall time: 1min 40s


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [48]:
y_pred_class_svm = clfrSVM.predict(X_test_dtm)

In [49]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class_svm)

0.92479359971126518

In [50]:
print(classification_report(y_test, y_pred_class_svm, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.97      0.94      0.95     16783
         과학       0.79      0.65      0.72       371
         국제       0.89      0.99      0.94     12136
         문화       0.90      0.88      0.89      3665
         사회       0.90      0.91      0.91     17451
         생활       0.81      0.63      0.71      1256
         정치       0.95      0.91      0.93     14835

avg / total       0.93      0.92      0.92     66497



https://stackoverflow.com/questions/40115043/no-space-left-on-device-error-while-fitting-sklearn-model

In [51]:
import numpy as np
counts = np.bincount(y_pred_class[(y_pred_class!=2) & (y_test==2)])
counts

array([297,  52,   0,  76, 197,  55, 110])

In [52]:
X_test[(y_pred_class==0) & (y_test==2)]

134870           아시아인프라투자은행(AIIB) 개소식에서 연설하는 리커창 중국 총리  ...
184399              1년만에 만장일치 결정 "노동시장 호조·물가상승 고려한 조치, 경...
314134           국제유가는 10일(현지시간) 비교적 큰 폭의 오름세를 보였다.     ...
31614            중국인, 미국 내 외국인 주택 거래 '큰손'으로       지난해 미국...
196063           런던 남동부의 주택가   "세 사는 가구 늘고 임차료도 올라 저축은커녕...
129654           한적한 중국 베이징의 한 백화점       중국의 지난해 재정수입 증가...
406737          스마트폰에 엇갈린 희비…대만은 작년 아이폰7 덕에 GDP 깜짝 성장   ...
217240           일본 주식 시장   외국인투자 증가 주요인…25년 연속 세계 최대 채권...
176502           중국 창업의 산실 중관춘［EPA=연합뉴스 자료사진］       중국에서...
196248                  영국의 유럽연합(EU) 이탈(브렉시트·Brexit) 결정 ...
99105                유럽 주요 증시는 18일(현지시간) 상승세로 출발했다.     ...
443258          "韓기업들 중국인 기호와 환경 적응 못해 경쟁력 상실" 주장  사드 보복...
261208           중국 저장성 항저우   홍콩 부동산 거래량 6개월간 39% 급감…25년...
301044           456억달러로 2015년과 비교해 두 배 늘어   중국의 대(對)미국 ...
91051           FT "1월 양적완화 발표로 금융시장 분위기 급반전"       '유로존...
298385            러시아가 우크라이나 사태 등을 둘러싼 서방과의 갈등으로 대다수 서방국...
111006           리커창 중국 총리.   "기본토대 개선·거

In [26]:
df[df.index==149282]

,article,title,article_len,sec1,sec_idx
149282,"기준금리 0.5% 유지…""주택 시장 둔화·수출 부진"" 캐나다 중앙은...","캐나다 중앙은행, 올 성장률 전망 1.1%로 낮춰",899,국제,2


In [53]:
with open("yonhap_svm_clf.pkl", 'wb') as handle:
    pickle.dump(clfrSVM, handle)

In [54]:
with open("yonhap_nb_clf.pkl", 'wb') as handle:
    pickle.dump(nb, handle)